In [1]:
import os, sys
import h5py
import numpy as np
from scipy.io import loadmat
import cv2
import matplotlib
%matplotlib inline 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy import matrix as mat
from sympy import *

In [9]:
# 定义函数变量x
x = Symbol('x')
y = Symbol('y')

# 对函数sin(x)求导，并且显示
print(diff(sin(x)+sin(x*y), y))

x*cos(x*y)


In [74]:
# 用参数表示K，R矩阵
f, px, py = symbols('f px py')
#r11, r12, r13, r21, r22, r23, r31, r32, r33 = symbols('r11 r12 r13 r21 r22 r23 r31 r32 r33')
#R = Matrix([[r11, r12, r13], [r21, r22, r23], [r31, r32, r33]])
w, x, y, z = symbols('w x y z')
R = Matrix([[1-2*(y*y+z*z), 2*(x*y-z*w), 2*(x*z+y*w)], [2*(x*y+z*w), 1-2*(x*x+z*z), 2*(y*z-x*w)], [2*(x*z-y*w), 2*(y*z+x*w), 1-2*(x*x+y*y)]])
K = Matrix([[f, 0, px], [0, f, py], [0, 0, 1]])
tx, ty, tz = symbols('tx ty tz')
T = Matrix([[tx], [ty], [tz]])
#print(KR)

# 用参数表示ABCDEF六个点坐标
a, b, c, th, al = symbols('a b c th al')
XA = Matrix([[-a * cos(th) * cos(al)], [c-a * sin(th)], [-a * cos(th) * sin(al)]])   
XB = Matrix([[0], [c], [0]])
XC = Matrix([[a * cos(th) * cos(al)], [c+a * sin(th)], [a * cos(th) * sin(al)]])
XD = Matrix([[-b], [0], [0]])
XE = Matrix([[0], [0], [0]])
XF = Matrix([[b], [0], [0]])

# 用参数表示u，v，w，f(A,B,C,D,E,F)
#u = Matrix([[(f*r11 + px*r31)*(-a*cos(al)*cos(th) + tx) + (f*r12 + px*r32)*(-a*sin(th) + c + ty) + (f*r13 + px*r33)*(-a*sin(al)*cos(th) + tz)]])
#v = Matrix([[(f*r21 + py*r31)*(-a*cos(al)*cos(th) + tx) + (f*r22 + py*r32)*(-a*sin(th) + c + ty) + (f*r23 + py*r33)*(-a*sin(al)*cos(th) + tz)]])
#w = Matrix([[r31*(-a*cos(al)*cos(th) + tx) + r32*(-a*sin(th) + c + ty) + r33*(-a*sin(al)*cos(th) + tz)]])
ua, va, wa, ub, vb, wb, uc, vc, wc, ud, vd, wd, ue, ve, we, uf, vf, wf = symbols('ua va wa ub vb wb uc vc wc ud vd wd ue ve we uf vf wf')
fa,fb,fc,fd,fe,ff = symbols('fa fb fc fd fe ff')
ua, va, wa = K[0,:] * (R * XA + T), K[1,:] * (R * XA + T), K[2,:] * (R * XA + T)
ub, vb, wb = K[0,:] * (R * XB + T), K[1,:] * (R * XB + T), K[2,:] * (R * XB + T)
uc, vc, wc = K[0,:] * (R * XC + T), K[1,:] * (R * XC + T), K[2,:] * (R * XC + T)
ud, vd, wd = K[0,:] * (R * XD + T), K[1,:] * (R * XD + T), K[2,:] * (R * XD + T)
ue, ve, we = K[0,:] * (R * XE + T), K[1,:] * (R * XE + T), K[2,:] * (R * XE + T)
uf, vf, wf = K[0,:] * (R * XF + T), K[1,:] * (R * XF + T), K[2,:] * (R * XF + T)
fa = Matrix([ua/wa, va/wa])
fb = Matrix([ub/wb, vb/wb])
fc = Matrix([uc/wc, vc/wc])
fd = Matrix([ud/wd, vd/wd])
fe = Matrix([ue/we, ve/we])
ff = Matrix([uf/wf, vf/wf])
print(fa)

Matrix([[(f*(-a*(2*w*y + 2*x*z)*sin(al)*cos(th) - a*(-2*y**2 - 2*z**2 + 1)*cos(al)*cos(th) + tx + (-a*sin(th) + c)*(-2*w*z + 2*x*y)) + px*(-a*(-2*w*y + 2*x*z)*cos(al)*cos(th) - a*(-2*x**2 - 2*y**2 + 1)*sin(al)*cos(th) + tz + (-a*sin(th) + c)*(2*w*x + 2*y*z)))/(-a*(-2*w*y + 2*x*z)*cos(al)*cos(th) - a*(-2*x**2 - 2*y**2 + 1)*sin(al)*cos(th) + tz + (-a*sin(th) + c)*(2*w*x + 2*y*z))], [(f*(-a*(-2*w*x + 2*y*z)*sin(al)*cos(th) - a*(2*w*z + 2*x*y)*cos(al)*cos(th) + ty + (-a*sin(th) + c)*(-2*x**2 - 2*z**2 + 1)) + py*(-a*(-2*w*y + 2*x*z)*cos(al)*cos(th) - a*(-2*x**2 - 2*y**2 + 1)*sin(al)*cos(th) + tz + (-a*sin(th) + c)*(2*w*x + 2*y*z)))/(-a*(-2*w*y + 2*x*z)*cos(al)*cos(th) - a*(-2*x**2 - 2*y**2 + 1)*sin(al)*cos(th) + tz + (-a*sin(th) + c)*(2*w*x + 2*y*z))]])


In [122]:
# 写Jacobian Matrix A矩阵每个点要写一个，B矩阵每个点也要写一个
# R_q四元数的J矩阵公用
args = Matrix([w, x, y, z])
R_q1 = R[0,:].jacobian(args)
R_q2 = R[1,:].jacobian(args)
R_q3 = R[2,:].jacobian(args)
R_q = Matrix([R_q1, R_q2, R_q3])   # 9 by 4 matrix

# JAA, JAB, JAC, JAD, JAE, JAF
# JAA(fa_q, fa_t)
fa_q1 = fa[0,:].jacobian(args)   # 1 by 4 matrix
fa_q2 = fa[1,:].jacobian(args)
fa_q = Matrix([fa_q1, fa_q2])
fa_t1 = fa[0,:].jacobian(Matrix([tx, ty, tz]))   # 1 by 3 matrix
fa_t2 = fa[0,:].jacobian(Matrix([tx, ty, tz]))
fa_t = Matrix([fa_t1, fa_t2])
JAA = Matrix([fa_q.T, fa_t.T]).T   # 2 by 7 matrix
# JAB
fb_q1 = fb[0,:].jacobian(args)   # 1 by 4 matrix
fb_q2 = fb[1,:].jacobian(args)
fb_q = Matrix([fb_q1, fb_q2])
fb_t1 = fb[0,:].jacobian(Matrix([tx, ty, tz]))   # 1 by 3 matrix
fb_t2 = fb[0,:].jacobian(Matrix([tx, ty, tz]))
fb_t = Matrix([fb_t1, fb_t2])
JAB = Matrix([fb_q.T, fb_t.T]).T   # 2 by 7 matrix
# JAC
fc_q1 = fc[0,:].jacobian(args)   # 1 by 4 matrix
fc_q2 = fc[1,:].jacobian(args)
fc_q = Matrix([fc_q1, fc_q2])
fc_t1 = fc[0,:].jacobian(Matrix([tx, ty, tz]))   # 1 by 3 matrix
fc_t2 = fc[0,:].jacobian(Matrix([tx, ty, tz]))
fc_t = Matrix([fc_t1, fc_t2])
JAC = Matrix([fc_q.T, fc_t.T]).T   # 2 by 7 matrix
# JAD
fd_q1 = fd[0,:].jacobian(args)   # 1 by 4 matrix
fd_q2 = fd[1,:].jacobian(args)
fd_q = Matrix([fd_q1, fd_q2])
fd_t1 = fd[0,:].jacobian(Matrix([tx, ty, tz]))   # 1 by 3 matrix
fd_t2 = fd[0,:].jacobian(Matrix([tx, ty, tz]))
fd_t = Matrix([fd_t1, fd_t2])
JAD = Matrix([fd_q.T, fd_t.T]).T   # 2 by 7 matrix
# JAE
fe_q1 = fe[0,:].jacobian(args)   # 1 by 4 matrix
fe_q2 = fe[1,:].jacobian(args)
fe_q = Matrix([fe_q1, fe_q2])
fe_t1 = fe[0,:].jacobian(Matrix([tx, ty, tz]))   # 1 by 3 matrix
fe_t2 = fe[0,:].jacobian(Matrix([tx, ty, tz]))
fe_t = Matrix([fe_t1, fe_t2])
JAE = Matrix([fe_q.T, fe_t.T]).T   # 2 by 7 matrix
# JAF
ff_q1 = ff[0,:].jacobian(args)   # 1 by 4 matrix
ff_q2 = ff[1,:].jacobian(args)
ff_q = Matrix([ff_q1, ff_q2])
ff_t1 = ff[0,:].jacobian(Matrix([tx, ty, tz]))   # 1 by 3 matrix
ff_t2 = ff[0,:].jacobian(Matrix([tx, ty, tz]))
ff_t = Matrix([ff_t1, ff_t2])
JAF = Matrix([ff_q.T, ff_t.T]).T   # 2 by 7 matrix
print(JAF[1,6])

px/(b*(-2*w*y + 2*x*z) + tz) - (f*(b*(-2*y**2 - 2*z**2 + 1) + tx) + px*(b*(-2*w*y + 2*x*z) + tz))/(b*(-2*w*y + 2*x*z) + tz)**2


In [145]:
# JBA, JBB, JBC, JBD, JBE, JBF
# JBA
Xargs = Matrix([a, b, c, th, al]) 
fa_X1 = fa[0,:].jacobian(Xargs)
fa_X2 = fa[1,:].jacobian(Xargs)
JBA = Matrix([fa_X1, fa_X2])   # 2 by 5 matrix
# JBB
fb_X1 = fb[0,:].jacobian(Xargs)
fb_X2 = fb[1,:].jacobian(Xargs)
JBB = Matrix([fb_X1, fb_X2])   # 2 by 5 matrix
# JBC
fc_X1 = fc[0,:].jacobian(Xargs)
fc_X2 = fc[1,:].jacobian(Xargs)
JBC = Matrix([fc_X1, fc_X2])   # 2 by 5 matrix
# JBD
fd_X1 = fd[0,:].jacobian(Xargs)
fd_X2 = fd[1,:].jacobian(Xargs)
JBD = Matrix([fd_X1, fd_X2])   # 2 by 5 matrix
# JBE
fe_X1 = fe[0,:].jacobian(Xargs)
fe_X2 = fe[1,:].jacobian(Xargs)
JBE = Matrix([fe_X1, fe_X2])   # 2 by 5 matrix
# JBF
ff_X1 = ff[0,:].jacobian(Xargs)
ff_X2 = ff[1,:].jacobian(Xargs)
JBF = Matrix([ff_X1, ff_X2])   # 2 by 5 matrix
print(JBF)

Matrix([[0, (f*(-2*y**2 - 2*z**2 + 1) + px*(-2*w*y + 2*x*z))/(b*(-2*w*y + 2*x*z) + tz) + (f*(b*(-2*y**2 - 2*z**2 + 1) + tx) + px*(b*(-2*w*y + 2*x*z) + tz))*(2*w*y - 2*x*z)/(b*(-2*w*y + 2*x*z) + tz)**2, 0, 0, 0], [0, (f*(2*w*z + 2*x*y) + py*(-2*w*y + 2*x*z))/(b*(-2*w*y + 2*x*z) + tz) + (f*(b*(2*w*z + 2*x*y) + ty) + py*(b*(-2*w*y + 2*x*z) + tz))*(2*w*y - 2*x*z)/(b*(-2*w*y + 2*x*z) + tz)**2, 0, 0, 0]])


In [146]:
# Simple Example
JAA_value = JAA.subs({f:1150, px:509, py:508, w:1, x:0, y:0, z:0, th:0, al:0, tx:-0.0061, ty:-0.1, tz:1, a:0.03, b:0.024, c:0.096})
JAB_value = JAB.subs({f:1150, px:509, py:508, w:1, x:0, y:0, z:0, th:0, al:0, tx:-0.0061, ty:-0.1, tz:1, a:0.03, b:0.024, c:0.096})
JAC_value = JAC.subs({f:1150, px:509, py:508, w:1, x:0, y:0, z:0, th:0, al:0, tx:-0.0061, ty:-0.1, tz:1, a:0.03, b:0.024, c:0.096})
JAD_value = JAD.subs({f:1150, px:509, py:508, w:1, x:0, y:0, z:0, th:0, al:0, tx:-0.0061, ty:-0.1, tz:1, a:0.03, b:0.024, c:0.096})
JAE_value = JAE.subs({f:1150, px:509, py:508, w:1, x:0, y:0, z:0, th:0, al:0, tx:-0.0061, ty:-0.1, tz:1, a:0.03, b:0.024, c:0.096})
JAF_value = JAF.subs({f:1150, px:509, py:508, w:1, x:0, y:0, z:0, th:0, al:0, tx:-0.0061, ty:-0.1, tz:1, a:0.03, b:0.024, c:0.096})
JA_value = Matrix([JAA_value,JAB_value,JAC_value,JAD_value,JAE_value,JAF_value])
print(JA_value)
JBA_value = JBA.subs({f:1150, px:509, py:508, w:1, x:0, y:0, z:0, th:0, al:0, tx:-0.0061, ty:-0.1, tz:1, a:0.03, b:0.024, c:0.096})
JBB_value = JBB.subs({f:1150, px:509, py:508, w:1, x:0, y:0, z:0, th:0, al:0, tx:-0.0061, ty:-0.1, tz:1, a:0.03, b:0.024, c:0.096})
JBC_value = JBC.subs({f:1150, px:509, py:508, w:1, x:0, y:0, z:0, th:0, al:0, tx:-0.0061, ty:-0.1, tz:1, a:0.03, b:0.024, c:0.096})
JBD_value = JBD.subs({f:1150, px:509, py:508, w:1, x:0, y:0, z:0, th:0, al:0, tx:-0.0061, ty:-0.1, tz:1, a:0.03, b:0.024, c:0.096})
JBE_value = JBE.subs({f:1150, px:509, py:508, w:1, x:0, y:0, z:0, th:0, al:0, tx:-0.0061, ty:-0.1, tz:1, a:0.03, b:0.024, c:0.096})
JBF_value = JBF.subs({f:1150, px:509, py:508, w:1, x:0, y:0, z:0, th:0, al:0, tx:-0.0061, ty:-0.1, tz:1, a:0.03, b:0.024, c:0.096})
JB_value = Matrix([JBA_value,JBB_value,JBC_value,JBD_value,JBE_value,JBF_value])
print(JB_value)

Matrix([[0, 7.97088000000001, 2.49090000000000, -220.800000000000, 1150, 0, 41.5150000000000], [0, 0.883200000000002, 0.276000000000003, -69.0000000000000, 1150, 0, 41.5150000000000], [0, 1.34688000000000, 0, -220.800000000000, 1150, 0, 7.01499999999999], [0, 0.883200000000002, 0, 0, 1150, 0, 7.01499999999999], [0, -5.27712000000000, 1.64910000000000, -220.800000000000, 1150, 0, -27.4850000000000], [0, 0.883200000000002, -0.276000000000003, 69.0000000000000, 1150, 0, -27.4850000000000], [0, 0, 1.66152000000000, 0, 1150, 0, 34.6150000000000], [0, 0, 5.52000000000000, -55.2000000000000, 1150, 0, 34.6150000000000], [0, 0, 0, 0, 1150, 0, 7.01499999999999], [0, 0, 0, 0, 1150, 0, 7.01499999999999], [0, 0, 0.988080000000000, 0, 1150, 0, -20.5850000000000], [0, 0, -5.52000000000000, 55.2000000000000, 1150, 0, -20.5850000000000]])
Matrix([[-1150, 0, 0, 0, -1.24545000000000], [0, 0, 1150, -34.5000000000000, -0.138000000000002], [0, 0, 0, 0, 0], [0, 0, 1150, 0, 0], [1150, 0, 0, 0, -0.824549999999

In [150]:
m = 1
n = 6
J = mat(np.zeros((m * n * 2, m * 7 + n * 5)))
for i in range(n):
    for j in range(m):
        J[2*(i+1)*(j+1)-2:2*(i+1)*(j+1), 7*j:7*j+7] = JA_value[2*i:2*i+2, :]
for i in range(n):
    for j in range(m):
        J[2*(i+1)*(j+1)-2:2*(i+1)*(j+1), m*7+i*5: m*7+i*5+5] = JB_value[2*i:2*i+2, :]
print(J)

[[ 0.00000e+00  7.97088e+00  2.49090e+00 -2.20800e+02  1.15000e+03
   0.00000e+00  4.15150e+01 -1.15000e+03  0.00000e+00  0.00000e+00
   0.00000e+00 -1.24545e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00]
 [ 0.00000e+00  8.83200e-01  2.76000e-01 -6.90000e+01  1.15000e+03
   0.00000e+00  4.15150e+01  0.00000e+00  0.00000e+00  1.15000e+03
  -3.45000e+01 -1.38000e-01  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
   0.00000e+00  0.00000e+00]
 [ 0